# 로컬 런타임을 이용해 렌더링
로컬 런타임을 이용해 Typst 파일을 렌더링할 수 있습니다.  

또, Typst 파일은 텍스트로 작성되므로 다양한 수단을 동원하여 여러 방면으로 활용할 수 있습니다.

## 1. 환경 구성하기

### 1.1. Typst 런타임 설치

Windows

In [ ]:
!winget install --id Typst.typst

macOS

In [ ]:
!brew install typst

Linux

리눅스 배포판과 패키지 매니저에 따라 패키지명과 설치 방법이 상이합니다. [Repology](https://repology.org/project/typst/versions)를 참조하세요.  

Ubuntu의 경우 Snap을 이용하여 간단하게 설치할 수 있습니다.

In [ ]:
!sudo snap install typst

### 1.2. `gdg-cnu/slide-typst` 클론하기

사전에 준비된 [`gdg-cnu/slide-typst`](https://github.com/gdg-cnu/slide-typst) 라이브러리를 클론하거나 링크에서 다운로드받아 이 경로에 위치시킵니다.  

`gdg-cnu/slide-typst`는 비공개 리포지토리이므로, 클론 명령을 사용하고자 한다면 인증 과정을 거쳐야합니다.

최신 버전의 Git이 설치된 환경이라면, 깃허브 로그인 없이 `gdg-cnu/slide-typst` 클론을 시도할 때 깃허브 로그인 창이 나타납니다.

그렇지 않을 경우 아래의 방법으로 직접 인증할 수 있습니다.

1. 방법 1: 개인용 액세스 토큰(PAT)을 발급받는 방법
    1. [깃허브 설정의 토큰 항목으로 이동합니다.](https://github.com/settings/tokens)
    2. 새 토큰을 발급받습니다.
    3. `https://<pat>@github.com/gdg-cnu/slide-typst.git` 주소로 클론합니다.  
        ```sh
        git clone https://<pat>@github.com/gdg-cnu/slide-typst.git
        ```
    - [더 알아보기](https://docs.github.com/ko/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens)
2. 방법 2: SSH 키를 등록하는 방법
    - [더 알아보기](https://docs.github.com/ko/authentication/keeping-your-account-and-data-secure/reviewing-your-ssh-keys)
3. 방법 3: [GitHub CLI](https://cli.github.com/)를 사용하는 방법
    1. GitHub CLI를 설치합니다.
    2. `gh auth login`으로 로그인을 진행합니다.
    3. `https://github.com/gdg-cnu/slide-typst.git` 주소로 클론합니다.  
        ```sh
        git clone https://github.com/gdg-cnu/slide-typst.git
        ```

이 때, 이 경로(`/using-local-runtime-with-jupyter`)에는 `lib` 폴더만 위치시키면 됩니다.  

따라서 &lt;방법 3&gt;에 따라 윈도우 PowerShell 환경에서 다음과 같이 수행할 수 있습니다:

In [ ]:
!gh auth login
!git clone https://github.com/gdg-cnu/slide-typst.git
!move slide-typst/lib ./

## 2. 타입스트 코드 믹스인

```typst
__PLACEHOLDER_TITLE_DATA__
```

[`main.typ`](./main.typ)에 치환자를 작성한 후, 파일 외부에서 이 치환자를 이용해 타이틀 코드를 삽입합니다.

`./main.typ`

```typst
#import "/lib/theme.typ"
#show: theme.theme.with()

#let ai-data = theme.slide.theme-color.green
#let dev = theme.slide.theme-color.blue
#let devrel = theme.slide.theme-color.red
#let game = theme.slide.theme-color.yellow

__PLACEHOLDER_TITLE_DATA__
```

In [3]:
from enum import Enum
class Theme(Enum):
    AI_DATA = 'green'
    DEV = 'blue'
    DEVREL = 'red'
    GAME = 'yellow'

class Thumbnail:
    def __init__(self, title: str, author: str, author_meta: str, author_image_path: str, theme: Theme):
        self.title = title
        self.author = author
        self.author_meta = author_meta
        self.author_image_path = author_image_path
        self.theme = theme

    @staticmethod
    def _render_content_linebreak(content: str) -> str:
        return content.replace('\n', '\\\n')

    def to_typst(self) -> str:
        '''
        Render as Typst Library API
        '''
        return '#theme.slide.title(\n' + \
                f'title: [{Thumbnail._render_content_linebreak(self.title)}],\n' + \
                f'author: "{self.author}",\n' + \
                f'author-meta: "{self.author_meta}",\n' + \
                f'author-image-path: "{self.author_image_path}",\n' + \
                f'theme: "{self.theme.value}"' + \
                ')'

In [4]:
thumbnails: list[Thumbnail] = [
    Thumbnail(
        '\n불량 메타버스 인디게임은\n게임 엔진의 꿈을 꾸는가',
        'Jonghyeon Park',
        'Chonnam National University',
        '/assets/profile01.jpg',
        Theme.GAME
    ),
    
]

In [5]:
rendered = '\n'.join(map(lambda each: each.to_typst(), thumbnails))

In [6]:
file = './main.typ'
placeholder = '__PLACEHOLDER_TITLE_DATA__'

typst = open(file, encoding='utf-8').read().replace(placeholder, rendered)
with open(file, 'w', encoding='utf-8') as f:
    f.write(typst)